**Code in this notebook adds trailing x-month average columns for select weather variables to the combined weather + fires dataset used in some of the team's modeling efforts.**

In [1]:
import pandas as pd
import numpy as np
import boto3 # import to make AWS S3 work as source of input and target of output
import io # import to make AWS S3 work as source of input and target of output
import warnings

_Reading in consolidated weather + fire data for overlapping period and states:_

In [2]:
noaa_on_fire = pd.read_csv('./data/mfi_df_yr.csv')

_Dropping redundant and not useful columns:_

In [3]:
noaa_on_fire.drop(columns = ['Unnamed: 0', 'time_burnt_fixed', 'fire_year', 'statecode', 'division'], inplace = True)
noaa_on_fire.columns

Index(['stat_cause_descr', 'fire_size', 'fire_size_class', 'state',
       'cont_date_fixed', 'disc_date_fixed', 'time_burnt', 'month', 'year',
       'yearmonth', 'pcp', 'tavg', 'pdsi', 'phdi', 'zndx', 'pmdi', 'cdd',
       'hdd', 'sp02', 'sp03', 'sp06', 'sp09', 'sp12', 'sp24', 'tmin', 'tmax'],
      dtype='object')

_Sorting data chronologically using the `yearmonth` column:_

In [4]:
noaa_on_fire.sort_values('yearmonth', inplace = True)

_Building a list of unique `yearmonth` indicators, sorted chronologically from oldest to newest:_

In [5]:
yrmth_sorted = list(noaa_on_fire['yearmonth'].unique()) 
max(yrmth_sorted)
min(yrmth_sorted) #based on the min and max, the dataset spans Jan-1992 to Dec-2015
len(yrmth_sorted) == (2016-1992)*12 # checking if all months are represented
# yrmth_sorted[(yrmth_sorted.index(199304) -12):yrmth_sorted.index(199304)] # this was a preview to validate the method

True

_Defining a function which takes in the number of months (X) and a variable available in the dataset and adds a new column for a trailing X months average of the specified variable:_

In [6]:
def trail_x_mth_avg(n_months, var_name):
    noaa_on_fire_t = pd.DataFrame() #initializing an empty df to which we'll append data with trailing avgs
    states = ['AZ', 'CA', 'CO','ID', 'NM','NV','MT','OR','UT','WA','WY']
    for state in states:
        temp_df = noaa_on_fire[noaa_on_fire['state'] == state] # creating a temporary datafile with only one state's data
        temp_list = [] # creating an empty list to temporarily store trailing averages in the same order as yrmth_sorted
        for yrmth in yrmth_sorted[:n_months]: 
            temp_list.append(np.nan) # the first n month's traling-n-month averages can't be calculated
        for yrmth in yrmth_sorted[n_months:]:  # below we average the values for each set of the n months after the first n:        
            if len(temp_df[temp_df['yearmonth'].isin(yrmth_sorted[(yrmth_sorted.index(yrmth)-n_months):yrmth_sorted.index(yrmth)])]) == 0:
                temp_list.append(temp_list[-1]) # if some month's data not available, repeat previous value (this didn't actualy fix the NaNs)        
            else:
                temp_list.append(np.mean(list(temp_df[temp_df['yearmonth'].isin(yrmth_sorted[(yrmth_sorted.index(yrmth)-n_months):yrmth_sorted.index(yrmth)])].groupby('yearmonth')[var_name].mean())))
                # if available, calculate appropriate average

                           #append a column with descriptive name to temporary dataframe:
        temp_df[str(var_name)+'_t'+str(n_months)+'m'] = temp_df['yearmonth'].apply(lambda x: temp_list[yrmth_sorted.index(x)])
        
        noaa_on_fire_t = pd.concat([noaa_on_fire_t,temp_df], axis = 0) # adding each state's data with trailing average column to the new dataset
        
    return noaa_on_fire_t # return the new dataframe

_Applying the function to climate variables for 12, 9, 6 and 3 months trailing averages:_

In [7]:
warnings.filterwarnings("ignore")

noaa_vars = ['tavg', 'pcp', 'pmdi', 'pdsi']
n_months = [3, 6, 9, 12]

for var in noaa_vars:
    for n in n_months:
        noaa_on_fire = trail_x_mth_avg(n, var)

_Checking that new variables are included in the dataset and are calculated correctly:_

In [8]:
noaa_on_fire.columns

Index(['stat_cause_descr', 'fire_size', 'fire_size_class', 'state',
       'cont_date_fixed', 'disc_date_fixed', 'time_burnt', 'month', 'year',
       'yearmonth', 'pcp', 'tavg', 'pdsi', 'phdi', 'zndx', 'pmdi', 'cdd',
       'hdd', 'sp02', 'sp03', 'sp06', 'sp09', 'sp12', 'sp24', 'tmin', 'tmax',
       'tavg_t3m', 'tavg_t6m', 'tavg_t9m', 'tavg_t12m', 'pcp_t3m', 'pcp_t6m',
       'pcp_t9m', 'pcp_t12m', 'pmdi_t3m', 'pmdi_t6m', 'pmdi_t9m', 'pmdi_t12m',
       'pdsi_t3m', 'pdsi_t6m', 'pdsi_t9m', 'pdsi_t12m'],
      dtype='object')

In [9]:
temp_state_yrmth = noaa_on_fire.groupby(['state', 'yearmonth'])[['tavg', 'tavg_t3m']].mean()
temp_state_yrmth.pivot_table(index = 'yearmonth', columns = 'state', values = ['tavg', 'tavg_t3m'])

tavg                                                        ...  \
state        AZ    CA    CO    ID    MT    NM    NV    OR    UT    WA  ...   
yearmonth                                                              ...   
199201     39.6  43.2   NaN  26.0  28.0  31.6  30.8  35.0   NaN  34.6  ...   
199202     47.1  49.1  31.9   NaN  32.3  39.4  39.3  40.9   NaN  39.6  ...   
199203     50.2  50.8  38.1  41.0  38.7  45.3  43.6  44.6  45.3  45.1  ...   
199204     61.2  58.9  48.2  45.7  43.8  54.6  52.9  48.1  54.6  47.6  ...   
199205     66.7  67.0  54.6  55.3  53.5  59.4  61.8  57.8  59.4  56.0  ...   
...         ...   ...   ...   ...   ...   ...   ...   ...   ...   ...  ...   
201508     80.6  76.5  66.9  65.8  65.9  73.4  72.7  66.9  73.4  66.6  ...   
201509     75.0  72.1  62.6  57.2  57.3  69.0  65.9  58.1  69.0  56.2  ...   
201510     64.5  65.0  51.4  50.7  48.0  57.5  56.2  54.3  57.5  52.4  ...   
201511     48.3  47.6  34.6  29.9  30.0  43.4   NaN  35.8  43.4  36.0  ...   
201512     40.9  42.3   NaN   NaN  23.6  35.9  30.8  32.4  35.9   NaN  ...   

            tavg_t3m                                                         \
state             CA         CO         ID         MT         NM         NV   
yearmonth                                                                     
199201           NaN        NaN        NaN        NaN        NaN        NaN   
199202           NaN        NaN        NaN        NaN        NaN        NaN   
199203           NaN        NaN        NaN        NaN        NaN        NaN   
199204     47.700000  35.000000  33.500000  33.000000  38.766667  37.900000   
199205     52.933333  39.400000  43.350000  38.266667  46.433333  45.266667   
...              ...        ...        ...        ...        ...        ...   
201508     70.066667  60.433333  60.533333  59.866667  67.666667  65.766667   
201509     75.300000  66.133333  65.200000  65.300000  72.700000  71.533333   
201510     74.533333  65.366667  62.700000  63.133333  71.833333  69.833333   
201511     71.200000  60.300000  57.900000  57.066667  66.633333        NaN   
201512     61.566667        NaN        NaN  45.100000  56.633333  61.050000   

                                                       
state             OR         UT         WA         WY  
yearmonth                                              
199201           NaN        NaN        NaN        NaN  
199202           NaN        NaN        NaN        NaN  
199203           NaN        NaN        NaN        NaN  
199204     40.166667  45.300000  39.766667  42.205000  
199205     44.533333  49.950000  44.100000  45.650000  
...              ...        ...        ...        ...  
201508     62.200000  67.666667  63.366667  68.710000  
201509     66.666667  72.700000  66.900000  72.746667  
201510     64.166667  71.833333  63.800000  71.803333  
201511     59.766667  66.633333  58.400000  66.243333  
201512     49.400000  56.633333        NaN  56.793333  

[288 rows x 22 columns]

_Saving updated dataset to a csv (the resulting file was huge, hence use of a shared AWS S3 bucket):_

In [10]:
#noaa_on_fire.to_csv('s3://git-to-amazon-s3-outputbucket-rorni8oehk4l/soulclimberchick/meteorology-fire-impact/data-files/mfi_df_yr_trail.csv')